In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader, random_split
import pandas as pd
from PIL import Image
import os


In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")


Using device: cpu


In [3]:
class SkinCancerStage2Dataset(Dataset):
    def __init__(self, csv_file, img_dir, transform=None):
        self.metadata = pd.read_csv(csv_file)
        self.img_dir = img_dir
        self.transform = transform

        # Keep only cancerous images
        cancerous_classes = ['mel', 'bcc', 'akiec']
        self.metadata = self.metadata[self.metadata['dx'].isin(cancerous_classes)]

        # Map:
        # mel = 1 (Melanoma)
        # bcc and akiec = 0 (Non-Melanoma)
        self.metadata['label'] = self.metadata['dx'].map({
            'mel': 1,
            'bcc': 0,
            'akiec': 0
        })

    def __len__(self):
        return len(self.metadata)

    def __getitem__(self, idx):
        img_name = self.metadata.iloc[idx]['image_id']
        img_path = os.path.join(self.img_dir, f"{img_name}.jpg")

        image = Image.open(img_path).convert("RGB")
        label = self.metadata.iloc[idx]['label']

        if self.transform:
            image = self.transform(image)

        return image, torch.tensor(label, dtype=torch.float32)

    

In [4]:
transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor()
])


In [5]:
csv_path = "C:/Users/Sujani/Desktop/FinalYearProject/training/data/HAM10000_metadata.csv"   
images_path = "C:/Users/Sujani/Desktop/FinalYearProject/training/data/HAM10000_images"       

dataset_stage2 = SkinCancerStage2Dataset(
    csv_file=csv_path,
    img_dir=images_path,
    transform=transform
)

train_size = int(0.8 * len(dataset_stage2))
test_size = len(dataset_stage2) - train_size
train_dataset_stage2, test_dataset_stage2 = random_split(dataset_stage2, [train_size, test_size])

train_loader_stage2 = DataLoader(train_dataset_stage2, batch_size=32, shuffle=True)
test_loader_stage2 = DataLoader(test_dataset_stage2, batch_size=32, shuffle=False)


In [6]:
class SkinCancerCNN(nn.Module):
    def __init__(self):
        super(SkinCancerCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 16 * 16, 128)
        self.fc2 = nn.Linear(128, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(-1, 64 * 16 * 16)
        x = torch.relu(self.fc1(x))
        x = self.sigmoid(self.fc2(x))
        return x

model_stage2 = SkinCancerCNN().to(device)


In [7]:
criterion_stage2 = nn.BCELoss()
optimizer_stage2 = optim.Adam(model_stage2.parameters(), lr=0.001)


In [8]:
num_epochs = 10

for epoch in range(num_epochs):
    model_stage2.train()
    running_loss = 0.0
    for inputs, labels in train_loader_stage2:
        inputs, labels = inputs.to(device), labels.to(device)
        labels = labels.unsqueeze(1)

        optimizer_stage2.zero_grad()
        outputs = model_stage2(inputs)
        loss = criterion_stage2(outputs, labels)
        loss.backward()
        optimizer_stage2.step()

        running_loss += loss.item()

    print(f"Stage 2 - Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader_stage2):.4f}")

print("Stage 2 Training completed!")


Stage 2 - Epoch [1/10], Loss: 0.5791
Stage 2 - Epoch [2/10], Loss: 0.3582
Stage 2 - Epoch [3/10], Loss: 0.3428
Stage 2 - Epoch [4/10], Loss: 0.2916
Stage 2 - Epoch [5/10], Loss: 0.3123
Stage 2 - Epoch [6/10], Loss: 0.2787
Stage 2 - Epoch [7/10], Loss: 0.2588
Stage 2 - Epoch [8/10], Loss: 0.2604
Stage 2 - Epoch [9/10], Loss: 0.2575
Stage 2 - Epoch [10/10], Loss: 0.2263
Stage 2 Training completed!


In [13]:
# Stage 2 Accuracy Checking
model_stage2.eval()

correct = 0
total = 0

with torch.no_grad():
    for inputs, labels in test_loader_stage2:
        inputs, labels = inputs.to(device), labels.to(device)
        labels = labels.unsqueeze(1)

        outputs = model_stage2(inputs)
        predicted = (outputs > 0.5).float()

        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy_stage2 = 100 * correct / total
print(f"Stage 2 Model Test Accuracy: {accuracy_stage2:.2f}%")


Stage 2 Model Test Accuracy: 85.68%


In [12]:
torch.save(model_stage2.state_dict(), "C:/Users/Sujani/Desktop/FinalYearProject/backend/model_stage2.pt")
print("Stage 2 Model saved successfully!")


Stage 2 Model saved successfully!
